In [1]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv
import os
import json

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
client = MongoClient(os.environ['MONGO_URI'])
db = client[os.environ['MONGO_DB']]
collection = db[os.environ['MONGO_COLLECTION']]
api_key = os.environ['API_KEY']

# Tek seferlik kullandım. Parquet dosyasını MongoDB'ye yüklemek için kullanıldı.

In [4]:
# # Read the Parquet file into a DataFrame
# df = pd.read_parquet(r'C:\Users\gkhne\Documents\GitHub\end-to-end-video-game-ml\notebooks\result.parquet')

# # Connect to MongoDB
# client = MongoClient(os.environ['MONGO_URI'])
# db = client[os.environ['MONGO_DB']]
# collection = db[os.environ['MONGO_COLLECTION']]
# # Convert DataFrame to a list of dictionaries
# data = df.to_dict(orient='records')

# # Insert the data into MongoDB
# collection.insert_many(data)

# # Close the MongoDB connection
# client.close()

In [5]:
def query_collection_by_year(year):
    # Query the collection for documents with released field starting with the specified year
    query = {'released': {'$regex': f'^{year}'}}
    cursor = collection.find(query)

    # Convert the cursor to a list of dictionaries
    data = list(cursor)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Print the DataFrame
    return df

In [6]:
df = query_collection_by_year('2019')

In [7]:
def query_collection_by_name(game_name):
    # Query the collection for documents with the specified game name
    query = {'name': {'$regex': f'^{game_name}'}}
    
    cursor = collection.find(query)

    # Convert the cursor to a list of dictionaries
    data = list(cursor)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Print the DataFrame
    return df

In [8]:
df = query_collection_by_name('Grand Theft Auto')
df = df[df['slug'] == "grand-theft-auto-v"]

In [9]:
import requests

def game_trailers(game_name):
    url = f"https://api.rawg.io/api/games/{game_name}/movies"
    params = {"key": api_key}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        movies = response.json()
        return movies['results'][0]['data']['480']
    else:
        return "Request failed with status code:", response.status_code


In [10]:
game_trailers("grand-theft-auto-v")


'https://steamcdn-a.akamaihd.net/steam/apps/256693661/movie480.mp4'